## 파인튜닝에 맞는 데이터 형태 변환

In [1]:
!pip install openai

In [2]:
pip install langchain_community langchain_openai faiss-cpu pypdf

Note: you may need to restart the kernel to use updated packages.


In [ ]:
## 동일 지문 안묶은 버전 
import json
from collections import defaultdict

# JSON 파일 로드
json_file_path = "./information_added.json"

with open(json_file_path, "r", encoding="utf-8") as file:
    data = json.load(file)


# 데이터 변환 함수
def transform_grouped_data(data):
    transformed_data = []
    
    for passage, items in data.items():
        subjects = list(set(item["subject"] for item in items))
        topics = list(set(topic for item in items for topic in item["topics"]))
        question_type = list(set(item["question_type"] for item in items))
        
        for item in items:
            question_content = item["question_text"] + " " + " ".join(
                [f"{choice['number']}. {choice['text']}" for choice in item["choices"]]
            )
            
            transformed_entry = {
                "messages": [
                    {"role": "system", "content": "당신은 대한민국 대학수학능력시험 국어영역 독서과목 지문과 문항을 출제하는 한국교육과정평가원 출제위원입니다. 본 시험은 2015 개정 국어과 교육과정을 바탕으로, 대학에서의 원만하고 능률적인 수학(修學)에 필요한 국어 능력을 측정하는 것을 목표로 합니다. 고등학교 국어과 교육과정 중 ‘독서’ 과목의 학습 목표와 내용을 중심으로 다양한 소재의 담화 및 글, 자료를 활용하여 학생들의 국어 능력을 측정한다는 목표가 지문 및 문항 구성에도 반영되어야 합니다. 지문은 인문･예술, 사회･문화, 과학･기술 등 다양한 분야를 아우르며, 설명문･논설문･보고서 등 여러 유형의 글을 활용합니다. 문항은 사실적･추론적･비판적･창의적 사고 능력을 종합적으로 평가하도록 구성합니다. 문항은 지문 내용을 정확히 이해했는지를 확인하는 것뿐만 아니라, 지문 속 정보와 개념을 활용하여 논리적으로 추론하거나 새로운 상황에 적용할 수 있는지 평가하는 방향으로 출제해야 합니다. 객관식 문항은 5지선다형으로 구성하되, 학생들이 피상적인 기억이 아니라 깊이 있는 이해와 사고를 통해 정답을 도출하도록 합니다."},
                    {"role": "user", "content": f"다음의 분야와 제재를 반영하여 대한민국 수학능력시험 국어영역 독서과목 문제 풀이를 위한 지문을 작성하세요. 분야 : {', '.join(subjects)}, 제재 : {', '.join(topics)}"},
                    {"role": "assistant", "content": passage},
                    {"role": "user", "content": f"위 지문에 대한 5지선다형 객관식 문제와 선택지를 문제 유형에 맞춰 생성하세요. 문제 유형 : {', '.join(question_type)}"},
                    {"role": "assistant", "content": question_content}
                ]
            }
            
            transformed_data.append(transformed_entry)
    
    return transformed_data

# 데이터 변환
converted_data = transform_grouped_data(data)

# 변환된 데이터 저장
output_file_path = "./finetuning_converted.json"
with open(output_file_path, "w", encoding="utf-8") as outfile:
    json.dump(converted_data, outfile, ensure_ascii=False, indent=4)

print(f"변환된 JSON 파일이 저장되었습니다: {output_file_path}")

In [16]:
## 동일 지문 묶은 버전
import json
from collections import defaultdict

# JSON 파일 로드
json_file_path = "./subject_topic_filtered.json"

with open(json_file_path, "r", encoding="utf-8") as file:
    data = json.load(file)

# passage를 기준으로 그룹화
grouped_data = defaultdict(list)
for item in data:
    grouped_data[item["passage"].strip()].append(item)

# passage별 그룹핑된 개수 확인
passage_group_counts = {passage: len(items) for passage, items in grouped_data.items()}


# 데이터 변환 함수
def transform_grouped_data(grouped_data):
    transformed_data = []
    for passage, items in grouped_data.items():
        subjects = list(set(item["subject"] for item in items))
        topics = list(set(topic for item in items for topic in item["topics"]))
        questions = []
        
        for item in items:
            question_content = item["question_text"] + " " + " ".join(
                [f"{choice['number']}. {choice['text']}" for choice in item["choices"]]
            )
            questions.append({"role": "user", "content": "위 지문에 대한 5지선다형 객관식 문제와 선택지를 생성하세요."})
            questions.append({"role": "assistant", "content": question_content, "weight": 1})
        
        transformed_entry = {
            "messages": [
                {"role": "system", "content": "당신은 대한민국 대학수학능력시험 국어영역 독서과목 지문과 문항을 출제하는 한국교육과정평가원 출제위원입니다. 본 시험은 2015 개정 국어과 교육과정을 바탕으로, 대학에서의 원만하고 능률적인 수학(修學)에 필요한 국어 능력을 측정하는 것을 목표로 합니다. 고등학교 국어과 교육과정 중 ‘독서’ 과목의 학습 목표와 내용을 중심으로 다양한 소재의 담화 및 글, 자료를 활용하여 학생들의 국어 능력을 측정한다는 목표가 지문 및 문항 구성에도 반영되어야 합니다. 지문은 인문･예술, 사회･문화, 과학･기술 등 다양한 분야를 아우르며, 설명문･논설문･보고서 등 여러 유형의 글을 활용합니다. 문항은 사실적･추론적･비판적･창의적 사고 능력을 종합적으로 평가하도록 구성합니다. 문항은 지문 내용을 정확히 이해했는지를 확인하는 것뿐만 아니라, 지문 속 정보와 개념을 활용하여 논리적으로 추론하거나 새로운 상황에 적용할 수 있는지 평가하는 방향으로 출제해야 합니다. 객관식 문항은 5지선다형으로 구성하되, 학생들이 피상적인 기억이 아니라 깊이 있는 이해와 사고를 통해 정답을 도출하도록 합니다."},
                {"role": "user", "content": f"다음의 분야와 제재를 반영하여 대한민국 수학능력시험 국어영역 독서과목 문제 풀이를 위한 지문을 작성하세요. 분야 : {', '.join(subjects)}, 제재 : {', '.join(topics)}"},
                {"role": "assistant", "content": passage, "weight": 0},
            ] + questions
        }
        transformed_data.append(transformed_entry)
    
    return transformed_data


# 데이터 변환
converted_data = transform_grouped_data(grouped_data)

# 변환된 데이터 저장
output_file_path = "./수능_모의고사_finetuning_converted.json"
with open(output_file_path, "w", encoding="utf-8") as outfile:
    json.dump(converted_data, outfile, ensure_ascii=False, indent=4)

print(f"변환된 JSON 파일이 저장되었습니다: {output_file_path}")


변환된 JSON 파일이 저장되었습니다: ./수능_모의고사_finetuning_converted.json
